# IX System Simulation - CLI Wrapper

This notebook executes ix_cli.py to ensure consistent results between all execution paths.

In [ ]:
# Parameters cell - papermill will inject values here
project_root = None
watertap_ix_transport_path = None
configuration = None
water_analysis = None
breakthrough_criteria = None
regenerant_parameters = None
acid_options = None
timestamp = None
simulation_options = {}

In [ ]:
import json
import subprocess
import tempfile
import os
import sys
from pathlib import Path

# Create config for CLI
cli_config = {
    "water_analysis": water_analysis,
    "configuration": configuration,
    "solver_options": simulation_options.get("solver_options", {})
}

# Write config to temp file
with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
    config_path = f.name
    json.dump(cli_config, f, indent=2)

with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
    output_path = f.name

In [ ]:
# Run CLI
cmd = [
    sys.executable,
    str(Path(project_root) / "ix_cli.py"),
    "run",
    config_path,
    "--output", output_path
]

result = subprocess.run(cmd, capture_output=True, text=True, cwd=project_root)

# Clean up config
os.unlink(config_path)

if result.returncode != 0:
    print("STDERR:", result.stderr)
    raise RuntimeError(f"CLI failed with code {result.returncode}")

In [ ]:
# Load and process results
with open(output_path, 'r') as f:
    cli_results = json.load(f)
os.unlink(output_path)

# Extract data from CLI results
sim_status = cli_results['simulation']['status']
treated_water = cli_results['simulation']['treated_water']
performance = cli_results['simulation']['performance']

# Build IX performance metrics
ix_performance = {}
for vessel_name in configuration['ix_vessels'].keys():
    vessel_config = configuration['ix_vessels'][vessel_name]
    bed_volume = vessel_config['resin_volume_m3']
    flow_rate = water_analysis['flow_m3_hr']
    
    # Estimate based on removal
    avg_removal = (performance['ca_removal_percent'] + performance['mg_removal_percent']) / 2
    breakthrough_bv = max(10, int(avg_removal * 10))
    
    ix_performance[vessel_name] = {
        'breakthrough_time_hours': breakthrough_bv * bed_volume / flow_rate,
        'bed_volumes_treated': breakthrough_bv,
        'regenerant_consumption_kg': bed_volume * 120,
        'average_hardness_leakage_mg_L': treated_water['ion_concentrations_mg_L'].get('Ca_2+', 0) * 2.5 + 
                                        treated_water['ion_concentrations_mg_L'].get('Mg_2+', 0) * 4.1,
        'capacity_utilization_percent': min(100, max(0, avg_removal))
    }

In [ ]:
# Format final results for MCP
results = {
    'status': sim_status,
    'watertap_notebook_path': 'cli_wrapper_execution',
    'model_type': 'watertap_cli',
    'actual_runtime_seconds': cli_results['simulation']['solve_time'],
    'treated_water': treated_water,
    'ix_performance': ix_performance,
    'degasser_performance': {},
    'water_quality_progression': [],
    'economics': {},
    'recommendations': [
        f"Ca removal: {performance['ca_removal_percent']:.1f}%",
        f"Mg removal: {performance['mg_removal_percent']:.1f}%",
        "Using ix_cli.py for consistent results"
    ],
    'detailed_results': {
        'solver_termination': cli_results['simulation']['solver_termination'],
        'degrees_of_freedom': cli_results['initialization']['dof'],
        'mass_balance_error': performance['mass_balance_error_percent']
    }
}

# This is what papermill will extract
results